In [ ]:
#install

!python -m spacy download en_core_web_md
!pip install wordcloud
!pip install pyspellchecker
!pip install contractions
!pip install -U textblob
!python -m textblob.download_corpora
!pip install empath
############################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pandas_profiling as pdp
import gc
gc.enable()
import spacy
import contractions
import en_core_web_md
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
from wordcloud import WordCloud
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import unicodedata
import re
import scattertext as st
from textblob import TextBlob
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
data = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.drop(["id","keyword","location"],axis=1,inplace=True)

# EDA

In [ ]:
reporte = pdp.ProfileReport(data, title="Pandas Profiling Reporte",minimal=True)
reporte

In [ ]:
data['contara'] = data["text"].apply(lambda x: len(str(x).split()))
data.head()

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(data = data, x = "target", y = "contara")
plt.title('Number of words in each target', fontsize= 25)
plt.xlabel('predator')
plt.ylabel('contara')

In [ ]:
sns.countplot(data=data,x='target',orient="h")
plt.title("Disaster and fake")
plt.xlabel("target")
plt.ylabel("count")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
data.target.value_counts().plot(kind="pie", autopct='%1.0f%%')
plt.ylabel("target")

plt.title("fake or not")
plt.show

# Preprocessing

## NLP and stopwords.

In [ ]:
nlp = en_core_web_md.load()

In [ ]:
stopwords_spacy = list(STOP_WORDS)
print(stopwords_spacy)
len(stopwords_spacy)

In [ ]:
en_stop_words=STOP_WORDS

## Spellcheck Activation.

In [ ]:
spell = SpellChecker(language='en',distance=1)

In [ ]:
def spell_check(x):
    correct_word = []
    mispelled_word = x
    for word in mispelled_word:
        correct_word.append(spell.correction(word))
    return ' '.join(correct_word)

In [ ]:
data['text']=data['text'].apply(str)

## Train

In [ ]:
%%time

pattern = r"[$&+,:;=_?@#|\[\]{}'<>.^*()%!-]"
tw_lemm=[]
lm = WordNetLemmatizer()
for i in range(data.shape[0]):
    tweet = data.iloc[i].text
    tweet = ' '.join(re.sub(pattern, '', tweet).strip().split())
    tweet = ' '.join([word for word in tweet.split() if word.isalpha()])
    tweet = tweet.lower()
    tweet = contractions.fix(tweet)
    tweet = unicodedata.normalize('NFKD', tweet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    tweet = ' '.join(word for word in tweet.split() if word not in en_stop_words)
    tweet = spell.correction(tweet)
    tweet = ' '.join([lm.lemmatize(word) for word in tweet.split()])
    tw_lemm.append(tweet)


In [ ]:
data["tweet_lemms"]=pd.Series(tw_lemm)

In [ ]:
data.head()

## Test

In [ ]:
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [ ]:
%%time

pattern = r"[$&+,:;=_?@#|\[\]{}'<>.^*()%!-]"
tw_test_lemm=[]
lm = WordNetLemmatizer()
for i in range(test.shape[0]):
    tweet = test.iloc[i].text
    tweet = ' '.join(re.sub(pattern, '', tweet).strip().split())
    tweet = ' '.join([word for word in tweet.split() if word.isalpha()])
    tweet = tweet.lower()
    tweet = contractions.fix(tweet)
    tweet = unicodedata.normalize('NFKD', tweet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    tweet = ' '.join(word for word in tweet.split() if word not in en_stop_words)
    tweet = spell.correction(tweet)
    tweet = ' '.join([lm.lemmatize(word) for word in tweet.split()])
    tw_test_lemm.append(tweet)

In [ ]:
test["tweet_lemms"]=pd.Series(tw_test_lemm)

## wordcloud.

In [ ]:
data_real = data[data.target == 1]
data_fake = data[data.target == 0]

In [ ]:
real = []
regexp=RegexpTokenizer(r"\w+")

for i in range(data_real.shape[0]):
    x = data_real.iloc[i].tweet_lemms
    x = regexp.tokenize(x) 
    x = [t for t in x if len(t)>1]
    real.append(x) 

In [ ]:
pred=[line for line in real for line in set(line)]
pred = Counter(pred)
pred = pred.most_common(20)
pred=pd.DataFrame(pred,columns = ['Words', 'Frequency'])
pred.head(10)

In [ ]:
fake = []
regexp=RegexpTokenizer(r"\w+")

for i in range(data_fake.shape[0]):
    x = data_fake.iloc[i].tweet_lemms
    x = regexp.tokenize(x) 
    x = [t for t in x if len(t)>1] 
    fake.append(x) 

In [ ]:
nopred=[line for line in fake for line in set(line)]
nopred = Counter(nopred)
nopred = nopred.most_common(20)
nopred=pd.DataFrame(nopred,columns = ['Words', 'Frequency'])
nopred.head(10)

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(321)
real=(" ").join(pred["Words"])
wc = WordCloud(max_words = 20, width = 1800 , height = 800 ).generate(real)
plt.imshow(wc)
plt.axis("off")
plt.title('Real disaster', fontsize=25)

plt.subplot(322)
fake=(" ").join(nopred["Words"])
wc = WordCloud(max_words = 20, width = 1800 , height = 800).generate(fake)
plt.imshow(wc)
plt.axis("off")
plt.title('fake disaster', fontsize=25)

plt.tight_layout()
plt.show()

## Scattertext.

In [ ]:
data2 = data.copy()

In [ ]:
data2['cat']=data['target'].astype("category").cat.rename_categories({0:'fake',1:'real'})

In [ ]:
%%time
scatter_corpus_nlp = st.CorpusFromPandas(data2,
                             category_col='cat',
                             text_col='tweet_lemms',nlp=nlp).build()

## Visualization of words in real / fake.

In [ ]:
html = st.produce_scattertext_explorer(scatter_corpus_nlp,
         category='real',category_name='real',         
        not_category_name='fake',width_in_pixels=1000,
          metadata=data2['cat'])
open("tweet-Visualization.html", 'wb').write(html.encode('utf-8'))
IFrame(src='tweet-Visualization.html', width = 1300, height=700)

## Visualization of specific terms but that have low occurrence compared to general terms.

In [ ]:
%%time

feat_builder = st.FeatsFromOnlyEmpath()
empath_corpus = st.CorpusFromParsedDocuments(data2,
                                              category_col='cat',
                                              feats_from_spacy_doc=feat_builder,
                                              parsed_col='tweet_lemms').build()
html = st.produce_scattertext_explorer(empath_corpus,
                                        category='real',
                                        category_name='real',
                                        not_category_name='fake',
                                        width_in_pixels=1000,
                                        metadata=data2['cat'],
                                        use_non_text_features=True,
                                        use_full_doc=True,
                                        topic_model_term_lists=feat_builder.get_top_model_term_lists())
open("tweet-Visualization-Empath.html", 'wb').write(html.encode('utf-8'))
IFrame(src='tweet-Visualization-Empath.html', width = 1300, height=700)

## Textblob.

In [ ]:
blob = TextBlob(str(data['tweet_lemms']))
pos_df = pd.DataFrame(blob.tags, columns = ['word' , 'pos'])
pos_df = pos_df.pos.value_counts()[:20]
pos_df.iplot(
    kind='bar',
    xTitle='POS',
    yTitle='count', 
    title='Top 12 Part-of-speech tagging para tweet_lemms')

### Train Test Split

In [ ]:
def counter_word (text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [ ]:
counter = counter_word(data["tweet_lemms"])

In [ ]:
vocab_size = len(counter)
embedding_dim = 32

max_length = 20
trunc_type='post'
padding_type='post'

oov_tok = "<XXX>"
seq_len = 12

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train = data["tweet_lemms"].values
X_test = test["tweet_lemms"].values
y_train = data.target

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Deep Learning.

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GaussianNoise
import keras

## Earlystopping.

In [ ]:
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=50,
    restore_best_weights=True
)

## LSTM.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.GaussianNoise(0.6),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.6),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.GaussianNoise(0.5),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 100
batch_size = 32

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.3,callbacks=[custom_early_stopping])

## Submission

In [ ]:
subm = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
preds = model.predict(X_test)
subm['target'] = (preds > 0.5).astype(int)
subm.to_csv("LSTM.csv", index=False, header=True)